In [1]:
import pandas as pd
import requests
from datetime import datetime
from tqdm import tqdm
import os

In [84]:
def getCountryDistrictValsHS(year,month):
    base = "https://api.census.gov/data/timeseries/intltrade/imports/hs"
    params = {"get":r','.join(["I_COMMODITY","I_COMMODITY_SDESC",
                                "CTY_CODE","CTY_NAME","DISTRICT",'DIST_NAME',
                                "GEN_VAL_MO","UNIT_QY1",
                                "CAL_DUT_MO","CNT_WGT_MO","CNT_VAL_MO",
                                "AIR_WGT_MO","AIR_VAL_MO"]),
                "CTY_NAME":['INDIA','CHINA','INDONESIA','VIETNAM'],
                "YEAR":year,
                "MONTH":month,
                "COMM_LVL":'HS10',
                "SUMMARY_LVL2":'HSCYDT'}
    temp = requests.get(base,params=params)
    df = pd.DataFrame(temp.json())
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df.columns = new_header #set the header row as the df header
    logTime = str(datetime.now().strftime(r"%Y_%m_%d-%I_%M_%S_%p"))
    fileName = f"apiResponses/{params['SUMMARY_LVL2']}_{year}_{month}_{logTime}.gzip"
    df.to_csv(fileName, compression='gzip')
    print(f"{fileName} Saved.! {df.shape} rows saved.!")
    # df = pd.DataFrame(temp[1:])
    return df

In [ ]:
def getCountryDistrictValsNACIS(year,month):
    base = "https://api.census.gov/data/timeseries/intltrade/imports/nacis"
    params = {"get":r','.join(["NAICS", "NAICS_LDESC", "NAICS_SDESC", 
                                "CTY_CODE","CTY_NAME","DISTRICT",'DIST_NAME',
                                "GEN_VAL_MO","UNIT_QY1",
                                "CAL_DUT_MO","CNT_WGT_MO","CNT_VAL_MO",
                                "AIR_WGT_MO","AIR_VAL_MO"]),
                "CTY_NAME":['INDIA','CHINA','INDONESIA','VIETNAM'],
                "YEAR":year,
                "MONTH":month,
                "COMM_LVL":'NA6',
                "SUMMARY_LVL2":'NACYDT'}
    temp = requests.get(base,params=params)
    df = pd.DataFrame(temp.json())
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df.columns = new_header #set the header row as the df header
    logTime = str(datetime.now().strftime(r"%Y_%m_%d-%I_%M_%S_%p"))
    fileName = f"apiResponses/{params['SUMMARY_LVL2']}_{year}_{month}_{logTime}.gzip"
    df.to_csv(fileName, compression='gzip')
    print(f"{fileName} Saved.! {df.shape} rows saved.!")
    # df = pd.DataFrame(temp[1:])
    return df

In [86]:
years = [str(year) for year in range(2022,2023)]
months = [str(month).zfill(2) for month in range(1,13)]
months

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

In [ ]:
for year in tqdm(years):
    for month in tqdm(months):
        getCountryDistrictValsHS(year,month)

In [2]:
listDf = []
for fileName in tqdm(os.listdir('apiResponses')):
    listDf.append(pd.read_csv(os.path.join('apiResponses',fileName),compression='gzip'))

hscydtDf = pd.concat(listDf)

100%|██████████| 60/60 [00:22<00:00,  2.65it/s]


In [13]:
hscydtDfIndia = hscydtDf[hscydtDf['CTY_NAME']=='INDIA']
hscydtDfIndia = hscydtDfIndia.drop('Unnamed: 0',axis=1)
hscydtDfIndia['HS2'] = hscydtDfIndia['I_COMMODITY'].apply(lambda x:str(x)[:2])
hscydtDfIndia['HS4'] = hscydtDfIndia['I_COMMODITY'].apply(lambda x:str(x)[:4])
hscydtDfIndia = hscydtDfIndia.rename({'I_COMMODITY':'HS6'},axis=1)
hscydtDfIndia['YearMonth'] = hscydtDfIndia[['YEAR','MONTH']].astype(str).agg('-'.join,axis=1)
hscydtDfIndiaFiltered = hscydtDfIndia[[,'HS2','HS4','HS6','I_COMMODITY_SDESC','DIST_NAME','GEN_VAL_MO','CNT_VAL_MO','AIR_VAL_MO','CAL_DUT_MO','CNT_WGT_MO','AIR_WGT_MO']]
hscydtDfIndiaFiltered

,HS2,HS4,HS6,I_COMMODITY_SDESC,DIST_NAME,GEN_VAL_MO,CNT_VAL_MO,AIR_VAL_MO,CAL_DUT_MO,CNT_WGT_MO,AIR_WGT_MO
3,19,1905,190590,"BREAD, PASTRY, CAKES, ETC NESOI & PUDDINGS","SAVANNAH, GA",469987,469987,0,9082,212576,0
5,19,1905,190590,"BREAD, PASTRY, CAKES, ETC NESOI & PUDDINGS","CHICAGO, IL",891104,891104,0,23331,400391,0
6,19,1905,190590,"BREAD, PASTRY, CAKES, ETC NESOI & PUDDINGS","HOUSTON-GALVESTON, TX",1202527,1202527,0,19238,576548,0
15,19,1905,190590,"BREAD, PASTRY, CAKES, ETC NESOI & PUDDINGS","BALTIMORE, MD",186715,186715,0,1370,87949,0
16,19,1905,190590,"BREAD, PASTRY, CAKES, ETC NESOI & PUDDINGS","NORFOLK, VA",117731,117731,0,0,82945,0
...,...,...,...,...,...,...,...,...,...,...,...
142846,73,7306,730661,"TUBES, PIPES ETC IRN/STL WLDD, SQ/RCT CR-SEC N...","CHICAGO, IL",89693,89693,0,0,27133,0
142847,73,7306,730661,"TUBES, PIPES ETC IRN/STL WLDD, SQ/RCT CR-SEC N...","CLEVELAND, OH",0,0,0,0,0,0
142848,73,7306,730661,"TUBES, PIPES ETC IRN/STL WLDD, SQ/RCT CR-SEC N...","SAN JUAN, PR",556342,556342,0,139087,627052,0
142849,73,7306,730661,"TUBES, PIPES ETC IRN/STL WLDD, SQ/RCT CR-SEC N...","MIAMI, FL",65516,65516,0,5316,68520,0


In [ ]:

import plotly.graph_objects as go

audi = df[df['car'] == 'Audi Q5']
benz = df[df['car'] == 'Merucedes GLE']
bmw = df[df['car'] == 'BMW X5']
x = df['date'].unique()

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=audi['Fuel Efficiency'],
                    mode='lines+markers',
                    name='Audi Q5', marker=dict(color="DarkOrange"))),
fig.add_trace(go.Scatter(x=x, y=benz['Fuel Efficiency'],
                    mode='lines+markers',
                    name='Merucedes GLE', marker=dict(color="Crimson"))),
fig.add_trace(go.Scatter(x=x, y=bmw['Fuel Efficiency'],
                    mode='lines+markers', name='BMW X5', marker=dict(color="RebeccaPurple")))

fig.update_layout(title_text='Fuel Efficiency Visual',
                 xaxis_title='Date', yaxis_title='FuelEfficiency')


fig.update_layout(
    updatemenus=[
        dict(active=0,
            buttons=list([
            dict(label="None",
                 method="update",
                 args=[{"visible":[True,True,True]},
                       {"title":"ALL"}]),
            dict(label="Audi Q5",
                 method="update",
                 args=[{"visible":[True, False, False]},
                       {"title":"Audi Q5"}]),
            dict(label="Merucedes GLE",
                 method="update",
                 args=[{"visible":[False,True,False]},
                       {"title":"Merucedes GLE"}]),
            dict(label="BMW X5",
                 method="update",
                 args=[{"visible":[False,False,True]},
                       {"title":"BMW X5"}])
        ]),
        )
    ]
)

fig.show()